# Install pyiqa

~/.local/share/ov/pkg/isaac_sim-2022.1.1/python.sh -m pip install git+https://github.com/chaofengc/IQA-PyTorch.git
~/.local/share/ov/pkg/isaac_sim-2022.1.1/python.sh -m pip install git+https://github.com/openai/CLIP.git

*(for original clip)

~/.local/share/ov/pkg/isaac_sim-2022.1.1/python.sh -m pip install transformers

In [1]:
import pyiqa
import torch

# list all available metrics
print(pyiqa.list_models())


['ahiq', 'brisque', 'ckdn', 'clipiqa', 'clipiqa+', 'cnniqa', 'cw_ssim', 'dbcnn', 'dists', 'fid', 'fsim', 'gmsd', 'ilniqe', 'lpips', 'lpips-vgg', 'mad', 'maniqa', 'ms_ssim', 'musiq', 'musiq-ava', 'musiq-koniq', 'musiq-paq2piq', 'musiq-spaq', 'nima', 'nima-vgg16-ava', 'niqe', 'nlpd', 'nrqm', 'paq2piq', 'pi', 'pieapp', 'psnr', 'psnry', 'ssim', 'ssimc', 'vif', 'vsi']


In [2]:
device = torch.device('cuda')

In [3]:
musiq_metric = pyiqa.create_metric('musiq-ava', device=torch.device('cuda'))
nima_vgg_metric = pyiqa.create_metric('nima-vgg16-ava', device=torch.device('cuda'))  # VGG16
nima_inception_metric = pyiqa.create_metric('nima', device=torch.device('cuda'))
clipiqa_metric = pyiqa.create_metric('clipiqa', device=torch.device('cuda'))

Loading pretrained model MUSIQ from /home/danny/.cache/torch/hub/checkpoints/musiq_ava_ckpt-e8d3f067.pth
Loading pretrained model NIMA from /home/danny/.cache/torch/hub/checkpoints/NIMA_VGG16_ava-dc4e8265.pth


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/inception_resnet_v2-940b1cd6.pth" to /home/danny/.cache/torch/hub/checkpoints/inception_resnet_v2-940b1cd6.pth


Downloading: "https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/NIMA_InceptionV2_ava-b0c77c00.pth" to /home/danny/.cache/torch/hub/checkpoints/NIMA_InceptionV2_ava-b0c77c00.pth



  0%|          | 0.00/208M [00:00<?, ?B/s]

Loading pretrained model NIMA from /home/danny/.cache/torch/hub/checkpoints/NIMA_InceptionV2_ava-b0c77c00.pth


100%|███████████████████████████████████████| 244M/244M [00:20<00:00, 12.7MiB/s]


In [10]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms

In [11]:
os.chdir("/home/danny/Desktop/omni-proj/DeepArrange/Example/0")

In [12]:
images = []
for img in os.listdir("./"):
    if img.find("png") != -1:
        images.append((img, Image.open(img).convert("RGB").resize((224, 224))))

In [13]:
to_tensor = transforms.Compose([transforms.ToTensor()])

In [14]:
img_tensors = [to_tensor(img[1]).unsqueeze(0) for img in images]

In [15]:
metrics = [musiq_metric, nima_vgg_metric, nima_inception_metric, clipiqa_metric]

In [24]:
scores = []
for metric in metrics:
    scores.append([metric(img).cpu().numpy()[0][0] for img in img_tensors])

In [31]:
pd.DataFrame(scores, 
             index=["musiq_metric", "nima_vgg_metric", "nima_inception_metric", "clipiqa_metric"],
             columns=[i[0] for i in images])

,4.png,dog.png,3.png,5.png,1.png,0.png,2.png
musiq_metric,4.029762,4.538571,3.769494,4.136200,3.350562,3.352724,3.629350
nima_vgg_metric,5.271444,5.511232,5.060489,5.133384,4.524152,4.499240,4.996006
nima_inception_metric,5.310588,6.047652,5.423373,5.218344,4.652073,4.631289,5.008500
clipiqa_metric,0.442904,0.656748,0.439789,0.321187,0.369269,0.359274,0.407500
